In [1]:
import numpy as np
import pandas as pd
import os

# File paths
technical_coeff_path = "C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/technical_coefficients/technical_coefficients_2019.csv"
price_volatility_path = "C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/price_data/II_PI_volatility.csv"
output_dir = "C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/results"

# Load data
technical_coefficients = pd.read_csv(technical_coeff_path, index_col=0)
price_volatility = pd.read_csv(price_volatility_path)
price_volatility.set_index('sector', inplace=True)  # New structure: 'sector' is the only index

# Sector mapping for standardization
sector_mapping = {
    "A01": "A01", "A02": "A02", "A03": "A03", "B": "B",
    "C10-C12": "C10T12", "C13-C15": "C13T15", "C16": "C16", "C17": "C17",
    "C18": "C18", "C19": "C19", "C20": "C20", "C21": "C21",
    "C22": "C22", "C23": "C23", "C24": "C24", "C25": "C25",
    "C26": "C26", "C27": "C27", "C28": "C28", "C29": "C29",
    "C30": "C30", "C31_C32": "C31_32", "C33": "C33", "D35": "D35",
    "E36": "E36", "E37-E39": "E37T39", "F": "F",
    "G45": "G45", "G46": "G46", "G47": "G47",
    "H49": "H49", "H50": "H50", "H51": "H51", "H52": "H52", "H53": "H53",
    "I": "I", "J58": "J58", "J59_J60": "J59_60", "J61": "J61", "J62_J63": "J62_63",
    "K64": "K64", "K65": "K65", "K66": "K66", "L68": "L",
    "M69_M70": "M69_70", "M71": "M71", "M72": "M72", "M73": "M73",
    "M74_M75": "M74_75", "N": "N", "O84": "O84", "P85": "P85",
    "Q": "Q", "R_S": "R_S", "T": "T", "U": "U"
}

# Function to map sector codes
def map_sector_code(sector):
    country_code, sector_code = sector.split('_', 1)
    if sector_code in sector_mapping:
        return f"{country_code}_{sector_mapping[sector_code]}"
    else:
        return sector  # Return unchanged if not in mapping

# Placeholder for results
unweighted_impacts = []

# Iterate over each sector in price_volatility
for sector, row in price_volatility.iterrows():
    price_shock = row['price_volatility']

    # Skip zero volatility or excluded sectors
    if price_shock == 0:
        continue

    # Standardize sector naming
    standardized_sector = map_sector_code(sector)
    exogenous_row = f"op_{standardized_sector}"
    exogenous_col = f"ip_{standardized_sector}"

    # Check if the row and column exist in the technical_coefficients DataFrame
    if exogenous_row not in technical_coefficients.index or exogenous_col not in technical_coefficients.columns:
        print(f"Skipping {sector} as it does not exist in technical_coefficients")
        continue

    # Extract A_EE (excluding the exogenous sector row and column) and transpose it
    A_EE = technical_coefficients.drop(index=[exogenous_row]).drop(columns=[exogenous_col]).T

    # Extract A_XE (coefficients of the exogenous sector influencing endogenous sectors)
    A_XE = technical_coefficients[exogenous_col].drop(exogenous_row).values.reshape(-1, 1)

    # Identity matrix for endogenous sectors
    I = np.eye(A_EE.shape[0])

    # Calculate Leontief inverse
    try:
        leontief_inverse_endogenous = np.linalg.inv(I - A_EE.values)
    except np.linalg.LinAlgError:
        print(f"Singular matrix encountered for {sector}")
        continue

    # Calculate ΔP_E using the modified Leontief price model
    delta_P_E = leontief_inverse_endogenous @ (A_XE * price_shock)

    # Store unweighted impacts directly in a DataFrame
    unweighted_impact_series = pd.Series(delta_P_E.flatten(), index=A_EE.index, name=standardized_sector)
    unweighted_impacts.append(unweighted_impact_series)

# Convert results to DataFrame
unweighted_df = pd.DataFrame(unweighted_impacts)

# Ensure columns are in the same order as technical_coefficients columns (for standardization)
column_order = technical_coefficients.columns  # Capture the original order of columns
unweighted_df = unweighted_df.reindex(columns=column_order)

# Save results to CSV
output_file = os.path.join(output_dir, 'unweighted_shock_impacts.csv')
unweighted_df.to_csv(output_file)

print(f"Unweighted impacts saved to {output_file}")


Skipping AT_A01 as it does not exist in technical_coefficients
Skipping AT_A02 as it does not exist in technical_coefficients
Skipping AT_A03 as it does not exist in technical_coefficients
Skipping AT_B as it does not exist in technical_coefficients
Skipping AT_C10-C12 as it does not exist in technical_coefficients
Skipping AT_C13-C15 as it does not exist in technical_coefficients
Skipping AT_C16 as it does not exist in technical_coefficients
Skipping AT_C17 as it does not exist in technical_coefficients
Skipping AT_C18 as it does not exist in technical_coefficients
Skipping AT_C19 as it does not exist in technical_coefficients
Skipping AT_C20 as it does not exist in technical_coefficients
Skipping AT_C21 as it does not exist in technical_coefficients
Skipping AT_C22 as it does not exist in technical_coefficients
Skipping AT_C23 as it does not exist in technical_coefficients
Skipping AT_C24 as it does not exist in technical_coefficients
Skipping AT_C25 as it does not exist in technical